In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# ZALO DATA

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
train_df = pd.read_json('/content/drive/MyDrive/TQA/code/zac2022_train_merged_final.json')

In [4]:
def get_keys(data):
    return [k for k, _ in data.items()]

def has_answer(df):
    return df[df.data.apply(get_keys).apply(lambda x: 'answer' in x)]

def get_context(data):
    return data['text']

def get_question(data):
    return data['question']

def get_answer_position(data):
    return data['short_candidate_start']

def get_answer(data):
    return data['short_candidate']

def question_context(df, i):
    question = df.data.apply(get_question)[i]
    text = test_df.context[i]
    print(
        f'- Question: {question}\n'
        f'- Context: {text}'
    )

In [5]:
questions = has_answer(train_df).data.apply(get_question).reset_index(drop=True)
contexts = has_answer(train_df).data.apply(get_context).reset_index(drop=True)
answers = has_answer(train_df).data.apply(get_answer).reset_index(drop=True)
answer_starts = has_answer(train_df).data.apply(get_answer_position).reset_index(drop=True)

In [6]:
zalo = pd.DataFrame(
    {
        'question': questions,
        'context': contexts,
        'answer': answers,
        'answer_start': answer_starts
    }
)

# Xsquad

In [7]:
df = pd.read_json('https://raw.githubusercontent.com/deepmind/xquad/master/xquad.vi.json')
df.drop('version', axis=1, inplace=True)

In [8]:
def get_paragraph(df):
    return df.data.apply(lambda x: x['paragraphs'])

In [9]:
para = get_paragraph(df)

dfs = []
for p in para:
    p_df = pd.DataFrame(p)
    question_df = p_df['qas'].apply(lambda x: x[0]['question']).to_frame('question')
    start_pos_and_text_df = p_df['qas'].apply(lambda x: x[0]['answers'][0]).apply(pd.Series)
    final_p_df = pd.concat([question_df, p_df, start_pos_and_text_df], axis=1)
    final_p_df.drop('qas', axis=1, inplace=True)
    final_p_df.columns = ['question', 'context', 'answer_start', 'answer']
    dfs.append(final_p_df)

In [10]:
xsquad = pd.concat(dfs, axis=0, ignore_index=True)
xsquad

,question,context,answer_start,answer
0,Đội thủ Panthers đã thua bao nhiêu điểm?,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",30,308
1,Ai thua đội Broncos ở vòng bảng?,Broncos đã đánh bại Pittsburgh Steelers ở vòng...,20,Pittsburgh Steelers
2,Peyton Manning bao nhiêu tuổi khi anh chơi tro...,Peyton Manning trở thành thủ quân (quaterback)...,196,39
3,Lady Gaga đã giành được bao nhiêu giải Grammy?,Người chiến thắng giải Grammy sáu lần và người...,30,sáu
4,Carolina đã bắt đầu ở vạch nào khi trận đấu cò...,"Khi thời gian quy định còn lại 4:51, Carolina ...",61,24
...,...,...,...,...
235,Ai đã đưa ra một cuộc thảo luận triết học về lực?,Aristotle đưa ra bản thảo luận triết học về kh...,0,Aristotle
236,Ai hình thành thuyết về lực vạn vật hấp dẫn?,Sự phát triển của các lý thuyết cơ bản đối với...,143,Isaac Newton
237,Lý thuyết nào giải thích rõ ràng nhất về trọng...,"Kể từ đó đến nay, thuyết tương đối đã được côn...",18,thuyết tương đối
238,Tốc độ thay đổi của điện tích là gì?,Bằng việc kết hợp định nghĩa của dòng diện giố...,33,dòng diện


# Bert QA

In [11]:
df = pd.read_json('https://raw.githubusercontent.com/mailong25/bert-vietnamese-question-answering/master/dataset/train-v2.0.json')

In [12]:
para = get_paragraph(df)

dfs = []
for p in tqdm(para):
    p_df = pd.DataFrame(p)
    p_df = p_df[p_df.qas.apply(lambda x: x[0]['answers'] != [])].reset_index(drop=True)
    if p_df.values.tolist() == []:
        continue
    question_df = p_df['qas'].apply(lambda x: x[0]['question']).to_frame('question')
    start_pos_and_text_df = p_df['qas'].apply(lambda x: x[0]['answers'][0]).apply(pd.Series)
    final_p_df = pd.concat([question_df, p_df, start_pos_and_text_df], axis=1)
    final_p_df.drop('qas', axis=1, inplace=True)
    final_p_df.columns = ['question', 'context', 'answer_start', 'answer']
    dfs.append(final_p_df)

  0%|          | 0/450 [00:00<?, ?it/s]

In [13]:
bert_qa = pd.concat(dfs, axis=0, ignore_index=True)

In [14]:
qa_dataset = pd.concat([zalo, xsquad, bert_qa], axis=0, ignore_index=True)

In [15]:
qa_dataset.to_json('qa_dataset.json')

# roberta-qa-fine-tuning

In [16]:
import os
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import multiprocessing as mp
from types import SimpleNamespace
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer
from torch.nn.parameter import Parameter
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from transformers import get_cosine_schedule_with_warmup
from transformers import AutoConfig, AutoModelForQuestionAnswering

N_CORES = mp.cpu_count()
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

### 1. General Settings

In [30]:
cfg = {
    'root': '/content/drive/MyDrive/TQA/code' + '/',
    'seed': 279,
    'batch_size': 16,
    'epochs': 30,
    'weight_decay': 1e-5,
    'learning_rate': 2e-5,
    'warmup_steps': 5,
    'max_length': 256,
    'intermediate_dropout': 0.3,
    'padding_quantile': 1.0,
    'device': 'cuda',
    'num_workers': N_CORES,
    'backbone': 'xlm-roberta-base'
}

cfg = SimpleNamespace(**cfg)

In [19]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def add_end_pos(df):
    len_answers = df.answer.apply(len)
    df['answer_end'] = df.answer_start + len_answers
    return df

def add_token_positions(encodings, start_pos, end_pos):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, start_pos[i]))
        end_positions.append(encodings.char_to_token(i, end_pos[i]-1))
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

@torch.inference_mode()
def run_prediction(context, question):
    inputs = tokenizer.encode_plus(
        question, context,
        return_tensors='pt'
    ).to(cfg.device)

    model.eval()
    with autocast():
        outputs = model(**inputs)

    answer_start = torch.argmax(outputs[0])
    answer_end = torch.argmax(outputs[1]) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(
            inputs.input_ids[0][answer_start:answer_end]
        )
    )

    print(
        f'- Question: {question}\n'
        f'- Answer: {answer}'
    )

### 2. Load in dataset

In [20]:
train_df = pd.read_json(cfg.root + 'qa_dataset.json')

### 3. Add end position

In [21]:
train_df = add_end_pos(train_df)

### 4. Tokenization

In [22]:
questions = train_df.question.values.tolist()
contexts = train_df.context.values.tolist()
answers = train_df.answer.values.tolist()
answer_starts = train_df.answer_start.values.tolist()
answer_ends = train_df.answer_end.values.tolist()

In [23]:
tokenizer = AutoTokenizer.from_pretrained(cfg.backbone)

encodings = tokenizer(
    contexts, questions,
    padding='max_length',
    truncation=True,
    max_length=cfg.max_length,
)

add_token_positions(
    encodings,
    answer_starts,
    answer_ends
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

### 5. Dataset and DataLoader

In [24]:
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [25]:
dataset = QADataset(encodings)
data_loader = DataLoader(
    dataset=dataset,
    batch_size=cfg.batch_size,
    drop_last=True,
    shuffle=True,
    num_workers=cfg.num_workers
)

### 6. Model Setups

In [26]:
config = AutoConfig.from_pretrained(cfg.backbone)
config.hidden_dropout_prob = cfg.intermediate_dropout
config.attention_probs_dropout_prob = cfg.intermediate_dropout

set_seed(seed=cfg.seed)
model = AutoModelForQuestionAnswering.from_pretrained(
    cfg.backbone,
    config=config
)
model.to(cfg.device);

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
optimizer = optim.AdamW(
    model.parameters(),
    lr=cfg.learning_rate,
    weight_decay=cfg.weight_decay
)

scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=cfg.warmup_steps,
    num_training_steps=len(data_loader)*cfg.epochs
)

scaler = GradScaler()

### 7. Fine-tuning

In [31]:
for epoch in tqdm(range(cfg.epochs)):

    model.train()

    for batch_idx, batch in enumerate(data_loader):
        input_ids = batch['input_ids'].to(cfg.device)
        attention_mask = batch['attention_mask'].to(cfg.device)
        start_positions = batch['start_positions'].to(cfg.device)
        end_positions = batch['end_positions'].to(cfg.device)

        idx = torch.quantile(
            torch.where(attention_mask==1)[1].float(),
            q=cfg.padding_quantile
        ).long()

        attention_mask = attention_mask[:, :idx]
        input_ids = input_ids[:, :idx]

        with autocast():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                start_positions=start_positions,
                end_positions=end_positions
            )

            loss = outputs[0]

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        optimizer.zero_grad()

        if not batch_idx % 100:
            print(
                f'Epoch: {epoch + 1}/{cfg.epochs}'
                f' | Batch: {batch_idx}/{len(data_loader)}'
                f' | Loss: {loss:.4f}'
            )

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1/30 | Batch: 0/388 | Loss: 5.4512
Epoch: 1/30 | Batch: 100/388 | Loss: 2.6255
Epoch: 1/30 | Batch: 200/388 | Loss: 3.1400
Epoch: 1/30 | Batch: 300/388 | Loss: 2.5083
Epoch: 2/30 | Batch: 0/388 | Loss: 2.1251
Epoch: 2/30 | Batch: 100/388 | Loss: 2.4791
Epoch: 2/30 | Batch: 200/388 | Loss: 1.7463
Epoch: 2/30 | Batch: 300/388 | Loss: 1.3961
Epoch: 3/30 | Batch: 0/388 | Loss: 0.9857
Epoch: 3/30 | Batch: 100/388 | Loss: 1.9190
Epoch: 3/30 | Batch: 200/388 | Loss: 1.1189
Epoch: 3/30 | Batch: 300/388 | Loss: 1.1627
Epoch: 4/30 | Batch: 0/388 | Loss: 1.1582
Epoch: 4/30 | Batch: 100/388 | Loss: 1.5140
Epoch: 4/30 | Batch: 200/388 | Loss: 1.0036
Epoch: 4/30 | Batch: 300/388 | Loss: 1.0655
Epoch: 5/30 | Batch: 0/388 | Loss: 0.7998
Epoch: 5/30 | Batch: 100/388 | Loss: 1.1492
Epoch: 5/30 | Batch: 200/388 | Loss: 1.5277
Epoch: 5/30 | Batch: 300/388 | Loss: 1.1410
Epoch: 6/30 | Batch: 0/388 | Loss: 1.1024
Epoch: 6/30 | Batch: 100/388 | Loss: 1.3218
Epoch: 6/30 | Batch: 200/388 | Loss: 1.3146


### 8. Save model

In [32]:
model.eval()
torch.save(model.state_dict(), 'qa_model.pth')

KeyboardInterrupt: 

In [33]:
import torch

# model là biến chứa mô hình của bạn
model.eval()  # Chuyển mô hình vào chế độ đánh giá

# Lưu trọng số mô hình
torch.save(model.state_dict(), '/content/drive/MyDrive/TQA/code/qa_model.pth')

# Q-A

In [34]:
context_1 = """
Elon Reeve Musk FRS (sinh ngày 28 tháng 6 năm 1971), là một kỹ sư, nhà tài phiệt,
nhà phát minh, doanh nhân công nghệ và nhà từ thiện người Mỹ gốc Nam Phi.
"""

context_2 = """
Elon Musk cùng với em trai, Kimbal, đồng sáng lập ra Zip2,
một công ty phần mềm web và được hãng Compaq mua lại với giá 340 triệu USD vào năm 1999.
"""

In [35]:
question_1 = 'Elon Musk là người nước nào?'
run_prediction(context_1, question_1)

- Question: Elon Musk là người nước nào?
- Answer: người Mỹ gốc Nam Phi


In [36]:
question_2 = 'Elon Musk sinh ngày bao nhiêu?'
run_prediction(context_1, question_2)

- Question: Elon Musk sinh ngày bao nhiêu?
- Answer: sinh ngày 28 tháng 6 năm 1971


In [37]:
question_3 = 'Em trai Elon tên là gì?'
run_prediction(context_2, question_3)

- Question: Em trai Elon tên là gì?
- Answer: Kimbal


In [38]:
question_4 = 'Hãng Compaq mua lại Zip2 với giá bao nhiêu?'
run_prediction(context_2, question_4)

- Question: Hãng Compaq mua lại Zip2 với giá bao nhiêu?
- Answer: 340 triệu USD
